<div style="display: flex; align-items: center; gap: 18px; margin-bottom: 15px;">
  <img src="https://files.codebasics.io/v3/images/sticky-logo.svg" alt="Codebasics Logo" style="display: inline-block;" width="130">
  <h1 style="font-size: 34px; color: #1f4e79; margin: 0; display: inline-block;">Codebasics Practice Room - Data Engineering Bootcamp </h1>
</div>

## 🧑🏼‍🔧 Setup

In [0]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, LongType, TimestampType
from delta.tables import DeltaTable

In [0]:
# --------------------------------------------
# ⚙️ Databricks Unity Catalog Setup (Auto)
# --------------------------------------------
from pyspark.sql import SparkSession

catalog_name = "practice_db_catalog"
schema_name = "clickstream"
volume_name = "data_volume"
folder_name = "clickstream"

# 1️⃣ Create Catalog if not exists
spark.sql(f"CREATE CATALOG IF NOT EXISTS {catalog_name}")
print(f"✅ Catalog `{catalog_name}` ready.")

# 2️⃣ Create Schema (Database) if not exists
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {catalog_name}.{schema_name}")
print(f"✅ Schema `{schema_name}` created inside `{catalog_name}`.")

# 3️⃣ Create Volume if not exists
spark.sql(f"CREATE VOLUME IF NOT EXISTS {catalog_name}.{schema_name}.{volume_name}")
print(f"✅ Volume `{volume_name}` created inside `{catalog_name}.{schema_name}`")

# 4️⃣ Set current context
spark.sql(f"USE CATALOG {catalog_name}")
spark.sql(f"USE {schema_name}")

# 5️⃣ Define volume-backed paths
base_path = f"/Volumes/{catalog_name}/{schema_name}/{volume_name}/{folder_name}"
raw_path = f"{base_path}/raw"


# 6️⃣ Create directories inside the volume
dbutils.fs.mkdirs(raw_path)

print("✅ Paths initialized successfully:")
print(f"Raw: {raw_path}")



In [0]:
# 🧮 Generate reproducible clickstream dataset (line-delimited JSON)
import json, random, datetime
from pathlib import Path

random.seed(42)  # ✅ reproducibility

# -----------------------------
# Configuration
# -----------------------------
num_users = 60
users = [f"U{u:03d}" for u in range(1, num_users + 1)]
event_types = ["view", "click", "scroll", "add_to_cart", "purchase"]
num_events = 700

# Destination in UC Volume
volume_path = "/Volumes/practice_db_catalog/clickstream/data_volume/clickstream/raw"
output_path = f"{volume_path}/events.json"

# Ensure directory exists
Path(volume_path).mkdir(parents=True, exist_ok=True)

# -----------------------------
# Data generation logic
# -----------------------------
current = datetime.datetime(2025, 1, 1, 8, 0, 0)
records = []

for _ in range(num_events):
    user = random.choice(users)
    gap_mins = random.randint(1, 120)  # variable user activity
    current += datetime.timedelta(minutes=gap_mins)
    records.append({
        "user_id": user,
        "event_time": current.strftime("%Y-%m-%dT%H:%M:%S"),
        "event_type": random.choice(event_types)
    })

# -----------------------------
# Write to Volume
# -----------------------------
with open(output_path, "w") as f:
    for r in records:
        json.dump(r, f)
        f.write("\n")

print(f"✅ Generated {len(records)} events for {len(users)} users → {output_path}")



# ❓ Scenario Question: Clickstream — Sessionize Events (PySpark) [Easy]




## 🗂️ Scenario

You are working with **raw website clickstream data** captured from multiple user sessions.  
Each event contains a `user_id`, a timestamp (`event_time`), and an `event_type`.  
Your task is to **sessionize** these events — grouping them into sessions based on user activity.

A **session** is defined as a sequence of consecutive user events where the **time gap between events is ≤ 30 minutes** for the same user.  
If there’s a gap of **> 30 minutes** between two events for the same user, a **new session** starts.

---

## 🎯 Task

Perform the following transformations using PySpark:

1. **Read** the input data from `/Volumes/practice_db_catalog/clickstream/data_volume/clickstream/raw/events.json`.  
2. **Order** events for each user by timestamp.  
3. **Identify** session boundaries using a **30-minute inactivity rule**.  
4. **Generate** a unique `session_id` for each user session.  
5. **Calculate** the `session_start` and `session_end` timestamps.  
6. **Display** the resulting DataFrame (`df_final`) with all sessionized events.

---

## 🧩 Assumptions

- Input file is line-delimited JSON at `/Volumes/practice_db_catalog/clickstream/data_volume/clickstream/raw/events.json`.  
- Timestamps are ISO 8601 strings (e.g., `2025-01-02T10:15:30`).  
- A user can have multiple sessions per day.  
- `session_id` can be created by combining `user_id` with an incremental session counter per user.

---

## 📦 Deliverables

- **Output Format:** Spark DataFrame (display via `.show()`)  
- **Expected Columns:**  
  `user_id`, `event_time`, `event_type`, `session_id`, `session_start`, `session_end`

---

## 🧠 Notes

- Use window functions: `lag`, `sum` (cumulative), `first`/`min`, `last`/`max`.  
- Time gap in minutes can be computed via `unix_timestamp` differences.  
- Keep transformations readable and well-commented.

---

## 🧩 Example Output (simplified)

| user_id | event_time          | event_type | session_id | session_start       | session_end         |
|---------|---------------------|------------|------------|---------------------|---------------------|
| U001    | 2025-01-01T10:00:00 | view       | U001_S1    | 2025-01-01T10:00:00 | 2025-01-01T10:20:00 |
| U001    | 2025-01-01T10:20:00 | click      | U001_S1    | 2025-01-01T10:00:00 | 2025-01-01T10:20:00 |
| U001    | 2025-01-01T11:10:00 | purchase   | U001_S2    | 2025-01-01T11:10:00 | 2025-01-01T11:10:00 |

---


## 🛢️Input data

In [0]:
df_raw = spark.read.json("/Volumes/practice_db_catalog/clickstream/data_volume/clickstream/raw/events.json")
display(df_raw.limit(10))

# 📝 Your Solution

In [0]:
# ✍️ Your Solution Here

from pyspark.sql import functions as F
from pyspark.sql.window import Window
# Steps:
# 1. Read the JSON file
# 2. Order events per user
# 3. Identify 30-minute session gaps
# 4. Generate session_id, session_start, session_end
# 5. Return df_final


## 🔍 Validation Questions

After creating the final DataFrame (`df_final`), answer these to check your understanding:

1. How many **sessions** does user **U010** have in total?  
2. What is the **average session duration (in minutes)** across all sessions?  
3. Which user has the **maximum number of sessions**? (return user_id and count)  
4. For user **U025**, what is the **start time** of their **first session**?  
5. How many **unique session IDs** were created in total?

In [0]:
#df_slvr = df_raw.groupBy("user_id").agg(F.collect_list("event_time").alias("event_time")).orderBy("event_time")
#df_slvr.limit(10).display()

In [0]:
#Defining Window Spec
window_spec = Window.partitionBy("user_id").orderBy("event_time")
#df_slvr = df_raw.withColumn("session_start",F.col("event_time"))
#df_slvr.limit(10).display()

In [0]:
#Adding session_start and session_end Coloums
df_slvr = df_raw.withColumns({"session_start" : 
                              F.when(F.timestamp_diff("MINUTE"
                                                      ,F.lag("event_time",1).over(window_spec)
                                                      ,"event_time") > 30 , F.col("event_time"))
                                    .otherwise(F.lag("event_time",1).over(window_spec)),
                                "session_end" : 
                              F.when(F.timestamp_diff("MINUTE"
                                                      ,"event_time"
                                                      ,F.lead("event_time",1).over(window_spec)) > 30, F.col("event_time"))
                                    .otherwise(F.lead("event_time",1).over(window_spec))    
                              })

df_slvr = df_slvr.withColumns({"session_start" : F.when(F.col("session_start").isNull(),F.col("event_time"))
                              .otherwise(F.col("session_start")),
                                "session_end" : F.when(F.col("session_end").isNull(),F.col("event_time"))
                              .otherwise(F.col("session_end"))         
                                         })                             

In [0]:
df_slvr = df_slvr.withColumn("session_length",F.timestamp_diff("MINUTE","session_start","session_end"))

In [0]:
df_final = df_slvr.withColumn(
    "session_id",
    F.when(
        (F.col("session_length") > 30) | (F.col("session_length") == 0) | (F.col("session_start") != F.lag("session_start",1).over(window_spec)), 
        F.concat(
            F.col("user_id"), 
            F.lit("-"), 
            F.row_number().over(window_spec).cast("string")
        )
    ).otherwise(None)
)
df_final = df_final.withColumn(
    "session_id",
    F.when(F.col("session_id").isNull(), F.lag("session_id",1).over(window_spec)).otherwise(F.col("session_id"))
)

In [0]:
df_final.filter(F.col("user_id") == "U002").orderBy("event_time").display()

In [0]:
df_final = df_final.select("user_id" ,"event_time","event_type","session_id","session_start","session_end","session_length")

In [0]:
df_final.display()

In [0]:
#How many sessions does user U010 have in total
df_final.filter(F.col("user_id") == "U010").orderBy("event_time").display()


In [0]:
#What is the average session duration (in minutes) across all sessions?
df_avg_session = df_final.groupBy("session_id").agg(F.avg("session_length"))
df_avg_session.agg(F.avg("avg(session_length)").cast("float")).display()

In [0]:
#Which user has the maximum number of sessions? (return user_id and count)
df_final.groupBy("user_id").agg(F.count("session_id")).orderBy(F.desc("count(session_id)")).display()

In [0]:
#For user U025, what is the start time of their first session
df_final.filter(F.col("user_id") == "U025").orderBy("event_time").display()

In [0]:
#How many unique session IDs were created in total?
df_final.select("session_id").distinct().count()
